In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
%pwd

'/home/alex/code/Korsas/final_project/mariajoi/flightdelay/notebooks'

In [2]:
%cd ../flightdelay/utils
from mytrans import MyTrans

/home/alex/code/Korsas/final_project/mariajoi/flightdelay/flightdelay/utils


/home/alex/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
df_test = pd.read_csv("../data/test_sample.csv")
df_test = df_test.drop("Unnamed: 0", axis = 1)


In [4]:
df_train = pd.read_csv("../data/train_sample.csv")
df_train = df_train.drop("Unnamed: 0", axis = 1)

In [5]:

X_test = df_test.drop(columns=['Tail_Number', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay',
             'Distance', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
       'ArrivalDelayGroups','CRSDepTime', 'CRSArrTime' , 'Cancelled', 'CancellationCode',
       'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'DepTime', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups',
        'DestCityName', 'DestState', 'OriginCityName', 'OriginState', 'DestAirportID',
     'Flight_Number_Operating_Airline', 'OriginAirportID','FlightDate','DayofMonth'])

X_train = df_train.drop(columns=['Tail_Number', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay',
             'Distance', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
       'ArrivalDelayGroups','CRSDepTime', 'CRSArrTime' , 'Cancelled', 'CancellationCode',
       'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'DepTime', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups',
        'DestCityName', 'DestState', 'OriginCityName', 'OriginState', 'DestAirportID',
     'Flight_Number_Operating_Airline', 'OriginAirportID','FlightDate','DayofMonth'])

In [6]:
df_train["BadFlight"] = ((df_train['ArrivalDelayGroups'] > 0) | (df_train['Cancelled'] > 0)).astype(int)
y_train = df_train["BadFlight"]

df_test["BadFlight"] = ((df_test['ArrivalDelayGroups'] > 0) | (df_test['Cancelled'] > 0)).astype(int)
y_test = df_test["BadFlight"]

In [7]:
trans = MyTrans()
trans.fit(X_train)
X_train_trans =trans.transform(X_train)

In [8]:
trans = MyTrans()
trans.fit(X_test)
X_test_trans =trans.transform(X_test)

In [9]:
preproc = pickle.load(open("../pickle/preproc_nottrans_notfit_.pkl","rb"))
preproc


ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa3937bff70>)])

# Model TPOT vs1

## Exporting to a pickle file

In [17]:
import os
from tpot import TPOTRegressor
from tpot import TPOTClassifier

model_reg = TPOTRegressor(generations=5, population_size=20, cv=5,random_state=42, verbosity=2)
model_classy = TPOTClassifier(generations=5, population_size=20, cv=5,random_state=42, verbosity=2)


In [18]:
# Add estimator

# Create and Fit Pipeline
model_pipeline = make_pipeline(preproc, model_classy)
model_pipeline
#model_pipeline.fit(X_train_trans,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa3937bff70>)])),
                ('tpotclassifier',
                 TPOTClassifier(generations=5, population_size=20,
                                random_state=42, verbosity=2))])

In [10]:
# %cd ../pickle
# # Export Pipeline as pickle file

# with open("fitted_model_pipeline_vs2.pkl", "wb") as file:
#     pickle.dump(model_pipeline, file)

/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


/Users/ronjaboehmer/code/mariajoi/flightdelay/flightdelay/pickle


## Training and testing scores

Using the balanced X_train and unbalanced y_train

In [ ]:
cross_val_score(model_pipeline, X_train_trans, y_train, cv=5, scoring='recall').mean()

In [24]:
preproc.fit(X_train_trans)
X_train_trans_t = pd.DataFrame(preproc.transform(X_train_trans))

/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [25]:
X_test_trans_t = pd.DataFrame(preproc.transform(X_test_trans))

In [26]:
model.fit(X_train_trans_t, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [28]:
y_pred = model.predict(X_test_trans_t)

In [65]:
y_pred_prob = model.predict_proba(X_test_trans_t)[:, 1]
y_pred2 = (y_pred_prob > 0.62).astype(int)

In [66]:
print('accuracy: %.3f' % accuracy_score(y_pred2, y_test))
print('f1_score: %.3f' % f1_score(y_pred2, y_test, average='binary'))
print('recall: %.3f' % recall_score(y_pred2, y_test,average='binary'))
print('precision: %.3f' % precision_score(y_pred2, y_test,average='binary'))
print("Confusion Matrix: ", confusion_matrix(y_pred2, y_test))


accuracy: 0.760
f1_score: 0.007
recall: 0.429
precision: 0.003
Confusion Matrix:  [[2732  861]
 [   4    3]]


In [29]:
print('accuracy: %.3f' % accuracy_score(y_pred, y_test))
print('f1_score: %.3f' % f1_score(y_pred, y_test, average='binary'))
print('recall: %.3f' % recall_score(y_pred, y_test,average='binary'))
print('precision: %.3f' % precision_score(y_pred, y_test,average='binary'))
print("Confusion Matrix: ", confusion_matrix(y_pred, y_test))


accuracy: 0.454
f1_score: 0.409
recall: 0.276
precision: 0.786
Confusion Matrix:  [[ 957  185]
 [1779  679]]


## Gradient boosting

In [70]:
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=42)
gb_clf.fit(X_train_trans_t, y_train)
y_preds_num = gb_clf.predict(X_test_trans_t)
print('accuracy: %.3f' % accuracy_score(y_preds_num,y_test))
print('f1_score: %.3f' % f1_score(y_preds_num,y_test, average='macro'))
print('recall: %.3f' % recall_score(y_preds_num,y_test,average='macro'))
print('precision: %.3f' % precision_score(y_preds_num,y_test,average='macro'))


print("Confusion Matrix: ", confusion_matrix(y_preds_num,y_test))


accuracy: 0.519
f1_score: 0.503
recall: 0.562
precision: 0.583
Confusion Matrix:  [[1261  255]
 [1475  609]]


In [77]:

# Instantiate model
gb_clf_grid = GradientBoostingClassifier(random_state=42)

param_grid_subset = {
    'n_estimators': [50, 100],
    'learning_rate': [0.1],
    'max_depth': [3, 4],
    'subsample': [0.8, 1.0]
}

# Hyperparameter Grid
grid_params = {'n_estimators': [50, 100, 200],                # Number of boosting stages to be run
                'learning_rate': [0.01, 0.1, 0.2],             # Step size shrinkage to prevent overfitting
                'max_depth': [3, 4, 5],                        # Maximum depth of the individual trees
                'min_samples_split': [2, 5, 10],               # Minimum samples required to split an internal node
                'min_samples_leaf': [1, 2, 4],                 # Minimum samples required to be at a leaf node
                'subsample': [0.8, 0.9, 1.0],                  # Fraction of samples used for fitting the trees
                'max_features': ['auto', 'sqrt', 'log2', None] # Number of features to consider for the best split
                }
scorers = {
  'precision_score': make_scorer(precision_score),
  'recall_score': make_scorer(recall_score),
  'accuracy_score': make_scorer(accuracy_score),
  'f1_score': make_scorer(f1_score)
}

# Instantiate Grid Search
search_gb = GridSearchCV(
    gb_clf_grid,
    param_grid_subset,
    scoring = "recall",
    cv = 5,
    n_jobs=-1 # parallelize computation
)

# Fit data to Grid Search
search_gb.fit(X_train_trans_t, y_train)
print(search_gb.best_estimator_)
print(search_gb.best_score_)
print(search_gb.best_params_)
print(search_gb.scorer_)


GradientBoostingClassifier(max_depth=4, n_estimators=50, random_state=42,
                           subsample=0.8)
0.6916444186648063
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'subsample': 0.8}
make_scorer(recall_score, average=binary)


In [79]:
gb_clf_grid = GradientBoostingClassifier(max_depth=4, n_estimators=50, random_state=42,
                           subsample=0.8)
gb_clf_grid.fit(X_train_trans_t, y_train)
y_preds_num = gb_clf_grid.predict(X_test_trans_t)
print('accuracy: %.3f' % accuracy_score(y_preds_num,y_test))
print('f1_score: %.3f' % f1_score(y_preds_num,y_test, average='macro'))
print('recall: %.3f' % recall_score(y_preds_num,y_test,average='macro'))
print('precision: %.3f' % precision_score(y_preds_num,y_test,average='macro'))


print("Confusion Matrix: ", confusion_matrix(y_preds_num,y_test))

accuracy: 0.556
f1_score: 0.528
recall: 0.566
precision: 0.590
Confusion Matrix:  [[1433  297]
 [1303  567]]


## Adaboost

In [74]:
ada_clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6, min_samples_leaf=10), learning_rate=0.1, random_state=42)
ada_clf.fit(X_train_trans_t, y_train)
y_preds_ada = ada_clf.predict(X_test_trans_t)

print('accuracy: %.3f' % accuracy_score(y_preds_ada,y_test))
print('f1_score: %.3f' % f1_score(y_preds_ada, y_test, average='binary'))
print('recall: %.3f' % recall_score(y_preds_ada, y_test,average='binary'))
print('precision: %.3f' % precision_score(y_preds_ada, y_test,average='binary'))


print("Confusion Matrix: ", confusion_matrix(y_preds_ada,y_test))


/Users/ronjaboehmer/.pyenv/versions/3.10.6/envs/flightdelay/lib/python3.10/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


accuracy: 0.566
f1_score: 0.404
recall: 0.301
precision: 0.611
Confusion Matrix:  [[1511  336]
 [1225  528]]
